In [ ]:
# Test cell

# The variables below have to be included in the test cell in order to run the notebook manually.
from init_sisense import sisense_conn
cube_name = "bank_churn"
additional_parameters = sisense_conn.load_additional_parameters(cube_name, table_name="bank_churn_train_auto_sklearn_ml")
# additional_parameters
sisense_conn.set_parameters(cube_name=cube_name, additional_parameters=additional_parameters)

# AutoML Training

In [ ]:
!pip install --upgrade sisense-automl
!pip install mpld3
!pip install ydata-profiling

In [ ]:
from sisense_automl import AutoMl
from datetime import datetime
import os
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from numpy import savetxt, loadtxt
pd.set_option('display.float_format', lambda x: '%.5f' % x)  ## to prevent scientific notation.
import joblib
from sklearn.metrics import confusion_matrix
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
)
import requests
import json

### Load input parameters from custom code

In [ ]:
table = sisense_conn.add_param["Dataset"]["table"]
objective = sisense_conn.add_param["Objective"]
target_column = sisense_conn.add_param["Target Column"]["column"]
drop_column = sisense_conn.add_param["Drop Feature"]

### Load data from Elasticube

In [ ]:
# Define Sql Statement
logical_sql1 = (f'SELECT * from [{table}] ')
print("SQL Statement:\n" + logical_sql1)
# Execute the SQL Statement
logical_sql_res1 = sisense_conn.get_logical_sql(query=logical_sql1, 
                                               cube_name=cube_name,  # passed to notebook from build / Test Cell
                                               count=None)  # limit the rows fetched

In [ ]:
# Known null representations needed since row from EC having null or empty values are represted as below in the list/array
null_representations = ['N\\A', 'NA', 'N/A', '']
column_names = logical_sql_res1['headers']
values = logical_sql_res1['values']
# Function to replace null representations
def replace_nulls(data, null_representations):
    return [[np.nan if v in null_representations else v for v in row] for row in data]

# Clean the data
values_cleaned = replace_nulls(values, null_representations)
# Get Data
df = pd.DataFrame(values_cleaned, columns = column_names)
if drop_column != '0':
    # Split the drop_column string into a list of column names
    columns_to_drop = drop_column.split(',')
    # Drop the specified columns
    df = df.drop(columns_to_drop, axis=1)
df.dtypes 

In [ ]:
df.head()

In [ ]:
## Create list of features for Widget
widget_features = df.columns.tolist()

# Remove the target column from the list
if target_column in widget_features:
    widget_features.remove(target_column)

### Create directory in Sisense to store files and model

In [ ]:
current_timestamp = datetime.now()
current_timestamp = current_timestamp.strftime('%Y%m%d%H%M%S')
folder_path = f"/opt/sisense/storage/notebooks/custom_code_notebooks/notebooks/automl/{table.split('.')[0]}/{current_timestamp}"
os.makedirs(folder_path, exist_ok=True)

### Store EDA lcoally

In [ ]:
profile = ProfileReport(df,
                        title="Exploratory Data Analysis",
                        explorative=True,
                        correlations={
                            "auto": {"calculate": True},
                            "pearson": {"calculate": True},
                            "spearman": {"calculate": True},
                            "kendall": {"calculate": True},
                            "phi_k": {"calculate": True},
                            "cramers": {"calculate": True},
                        })
profile.to_file(folder_path + '/eda.html')

# profile

### For large datasets or those with class imbalance (where one class has significantly more records than others), it’s important to train the model on a representative subset. Using stratified sampling ensures that the class distribution is preserved, improving data quality and model performance.

In [ ]:
# stratified_sample = df.groupby('Payment_Cat').apply(
#     lambda x: x.sample(frac=0.01)
# )

# # Removing the extra index added by groupby()
# stratified_sample = stratified_sample.droplevel(0)
# stratified_sample.head()
# stratified_sample.shape

# Start Model Training
### Call automl class

### Optional Parameters:

#### 1. `time_left_for_this_task`:
This parameter defines the total time allowed for the entire AutoML task, including feature selection, model training, hyperparameter optimization, and model ensemble creation.

- **Default**: `time_left_for_this_task=1800` (30 minutes).
- **Usage**: If not specified, the AutoML process will have 30 minutes to complete. Increase this for larger datasets or more complex models.

#### 2. `per_run_time_limit`:
This parameter defines the maximum time allowed for each individual model training and validation run. It ensures that no single model will take more than the specified time to train and evaluate.

- **Default**: `per_run_time_limit=360` (6 minutes).
- **Usage**: Adjust this based on the complexity of the models or dataset. Increasing this might be useful for deeper models that require more training time.

#### 3. `ensemble_size`:
The number of models to include in the final ensemble. The ensemble combines the best models to create a more robust and accurate final model.

- **Default**: `ensemble_size=5`.
- **Usage**: A higher ensemble size can improve model accuracy but may also increase the computational cost and model complexity.

#### 4. `n_jobs`:
This parameter defines the number of CPU cores used for parallel model training. It controls the degree of parallelism to speed up training.

- **Default**: `n_jobs=8`.
- **Usage**: Set this according to the number of available CPU cores on your machine. A higher value allows more models to be trained simultaneously.

### Summary of Default Values:
- **time_left_for_this_task**: `1800` (30 minutes).
- **per_run_time_limit**: `360` (6 minutes).
- **ensemble_size**: `5`.
- **n_jobs**: `8`.


In [ ]:
# # Using default parameters values:
# automl = AutoMl(df, target_column, objective, folder_path) 

# Using custom parameters for all four optional parameters
automl = AutoMl(df, target_column, objective, folder_path,
                time_left_for_this_task=3600,
                per_run_time_limit=360,
                ensemble_size=5,
                n_jobs=8)

#print(automl.file_name)

# Test Model Accuracy
### Import model

In [ ]:
model = automl.file_name
loaded_model = joblib.load(f"{folder_path}/{model}")

### Trained model ranking

In [ ]:
loaded_model.leaderboard()

### Import test data

In [ ]:
# load numpy array from csv file
X_test = pd.read_csv(f"{folder_path}/X_test.csv")
y_test = pd.read_csv(f"{folder_path}/y_test.csv")
y_test = y_test.to_numpy()

### Import Transformer Pipeline

In [ ]:
transformer_pipeline = joblib.load(f"{folder_path}/transformer_pipeline" )


### Apply feature engineering on test data

In [ ]:
X_test = transformer_pipeline.transform(X_test)

### Make prediction on test data

In [ ]:
pred =loaded_model.predict(X_test)
print(pred)

### Calculate Score

In [ ]:
score = {}
metrics = []
if objective.lower() == 'regression':
    # Root Mean Squared Error (RMSE)
    rmse_score = mean_squared_error(y_test, pred, squared=False)
    rmse_score = round(rmse_score, 2)
    print(f'RMSE: {rmse_score}')
    score['RMSE'] = rmse_score
    metrics.append(('RMSE', rmse_score))
    
    # Mean Absolute Error (MAE)
    mae_score = mean_absolute_error(y_test, pred)
    mae_score = round(mae_score, 2)
    print(f'MAE: {mae_score}')
    metrics.append(('MAE', mae_score))
    
    # R-squared (R2)
    r2 = r2_score(y_test, pred)
    r2 = round(r2, 2)
    print(f'R2 Score: {r2}')
    metrics.append(('R2', r2))
else:
    # Create confusion matrix
    cm = confusion_matrix(y_test, pred)

    # Visualize the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.ylabel('True Label', fontsize=13)
    plt.xlabel('Predicted Label', fontsize=13)
    plt.title('Confusion Matrix', fontsize=17)
    plt.show()
    
    # Save the plot as an HTML file
    html_file_path = f'{folder_path}/confusion_matrix.html'
    with open(html_file_path, 'w') as f:
        f.write(mpld3.fig_to_html(plt.gcf()))
    plt.close()
    
    # Calculate F1 score
    f1 = f1_score(y_test, pred, average='weighted')  # Use appropriate average method if needed
    f1 = round(f1, 2)
    print(f"F1 Score: {f1}")
    score['F1'] = f1
    metrics.append(('F1', f1))
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, pred)
    accuracy = round(accuracy, 2)
    print("Accuracy:", accuracy)
    metrics.append(('Accuracy', accuracy))
    
    # Calculate precision
    precision = precision_score(y_test, pred, average='weighted')
    precision = round(precision, 2)
    print("Precision:", precision)
    metrics.append(('Precision', precision))
    
    # Calculate recall
    recall = recall_score(y_test, pred, average='weighted')
    recall = round(recall, 2)
    print("Recall:", recall)
    metrics.append(('Recall', recall))
                    
# Create a DataFrame for all metrics
df_metrics = pd.DataFrame(metrics, columns=['MetricName', 'Value'])

# Save metrics to a CSV file
csv_file_path = os.path.join(folder_path, 'model_metrics.csv')
df_metrics.to_csv(csv_file_path, index=False)

score

### Create Dashbaord for Online Prediction

In [ ]:
dashboard_name = cube_name + '_' + table.split('.')[0] + '_auto_sklearn'
dashboard_name

In [ ]:
## Check if dashbaord exsists
url = 'http://' + os.environ['API_GATEWAY_EXTERNAL_SERVICE_HOST'] + ':' + os.environ['API_GATEWAY_EXTERNAL_SERVICE_PORT']
endpoint = f'/api/v1/dashboards?datasourceTitle={cube_name}'

response = sisense_conn.call_api_custom('GET',url,endpoint, params=None, payload=None)
response
res = response.json()
dashboard_exists = False
dash_id = ''
# Check if the dashboard already exists
for dash in res:
    if dash['title'] == dashboard_name:
        dashboard_exists = True
        dash_id = dash['oid']
        print(f"Dashboard '{dashboard_name}' - '{dash_id}' Already Exists")
        break

# If the dashboard does not exist, create it
if not dashboard_exists:
    print(f'Creating Dashboard as {dashboard_name}')
    payload = {
        "title": dashboard_name,
        "datasource": {
            "fullname": f"localhost/{cube_name}",
            "id": f"localhost_{cube_name}",
            "address": "LocalHost",
            "database": cube_name,
            "live": False,
            "title": cube_name
        },
        "type": "dashboard",
        "desc": "",
        "filters": [],
        "style": {},
        "editing": True
    }

    dash_endpoint = f'/api/dashboards/'
    # response = requests.post(dash_endpoint, headers=source_head, data=json.dumps(payload))
    response = sisense_conn.call_api_custom('POST',url,dash_endpoint, params=None, payload=payload)
    res=response.json()
    dash_id = res[0]['oid']
    
    if response.status_code == 200:
        print(f"Dashboard '{dashboard_name}' - '{dash_id}' created successfully")
    else:
        print(f'Failed to create dashboard {dashboard_name}. Status code: {response.status_code}')

In [ ]:
## Create Widget Payload 
def generate_payload(features):
    columns = []
    num_columns = 3
    items_per_column = len(features) // num_columns
    extra_items = len(features) % num_columns
    
    feature_index = 0

    for col in range(num_columns):
        column = {
            "type": "Column",
            "separator": col == 0,  # Only the first column has separator set to True
            "spacing": "large",
            "items": []
        }

        for item in range(items_per_column + (1 if col < extra_items else 0)):
            feature = features[feature_index]
            item_dict = {
                "spacing": "large",
                "type": "Input.Text",
                "id": f"data.{feature}",
                "placeholder": feature,
                "defaultValue": "",
                "isMultiline": True,
                "rows": "2",
                "borderRadius": "14px",
                "borderStyle": "none",
                "backgroundColor": "lightgrey"
            }
            column["items"].append(item_dict)
            feature_index += 1

        columns.append(column)

    payload = {
        "columns": columns
    }

    # return json.dumps(payload, indent=4)
    return payload



In [ ]:
## Check if widget exists
# widget_endpoint = f'{url}/api/v1/dashboards/{dash_id}/widgets'
# response = requests.get(widget_endpoint, headers=source_head)
widget_endpoint = f'/api/v1/dashboards/{dash_id}/widgets'
response = sisense_conn.call_api_custom('GET',url,widget_endpoint, params=None, payload=None)
res = response.json()
widget_id = ''
widget_exists = False

# Check if the widget already exists
for widget in res:
    if widget['title'] == table.split('.')[0] + '_auto_sklearn_online_prediction':
        widget_exists = True
        widget_id= widget["oid"]
        print(f"Widget {widget_id} 'Online-Prediction' in '{dashboard_name}' Already Exists")
        break

# If the widget does not exist, create it
if not widget_exists:
    print(f'Creating Widget in {dashboard_name}')
    payload = {
    "title": table.split('.')[0] + '_auto_sklearn_online_prediction',
    "type": "BloX",
    "subtype": "BloX",
    "desc": None,
    "source": None,
    "datasource": {
        "title": cube_name,
        "fullname": f'LocalHost/{cube_name}',
        "id": f'LOCALHOST_{cube_name}',
        "address": "LocalHost",
        "database": cube_name
    },
    "selection": None,
    "metadata": {
        "ignore": {
            "dimensions": [],
            "ids": [],
            "all": False
        },
        "panels": [
            {
                "name": "Items",
                "items": []
            },
            {
                "name": "Values",
                "items": []
            },
            {
                "name": "filters",
                "items": []
            }
        ],
        "usedFormulasMapping": {}
    },
    "style": {
        "currentCard": {
            "style": "",
            "script": "",
            "title": "",
            "showCarousel": True,
            "backgroundImage": "",
            "body": [
                {
                    "type": "Container",
                    "items": [
                        {
                            "type": "TextBlock",
                            "text": "Sisense AutoML",
                            "size": "extraLarge",
                            "color": "yellow",
                            "weight": "bold",
                            "horizontalAlignment": "center"
                        },
                        {
                            "type": "ColumnSet",
                            "spacing": "extraLarge",
                            "columns": []
                        }
                    ]
                },
                {
                    "type": "Container",
                    "separator": False,
                    "id": "outputContainer",
                    "size": "Large",
                    "items": [
                        {
                            "type": "ActionSet",
                            "actions": [
                                {
                                    "type": "new_online_prediction",
                                    "title": "Predict",
                                    "data": {
                                        "question": "",
                                        "table": ""
                                    }
                                }
                            ]
                        },
                        {
                            "spacing": "extraLarge",
                            "type": "TextBlock",
                            "text": "Output ",
                            "color": "green"
                        }
                    ]
                }
            ]
        },
        "currentConfig": {
            "fontFamily": "Open Sans",
            "fontSizes": {
                "default": 16,
                "small": 12,
                "medium": 22,
                "large": 32,
                "extraLarge": 50
            },
            "fontWeights": {
                "default": 500,
                "light": 100,
                "bold": 900
            },
            "containerStyles": {
                "default": {
                    "backgroundColor": "black",
                    "foregroundColors": {
                        "default": {
                            "normal": "#3A4356"
                        },
                        "white": {
                            "normal": "#ffffff"
                        },
                        "grey": {
                            "normal": "#dcdcdc"
                        },
                        "orange": {
                            "normal": "#f2B900"
                        },
                        "yellow": {
                            "normal": "#ffcb05"
                        },
                        "black": {
                            "normal": "#000000"
                        },
                        "lightGreen": {
                            "normal": "#93c0c0"
                        },
                        "green": {
                            "normal": "#54a254"
                        },
                        "red": {
                            "normal": "#dd1111"
                        },
                        "accent": {
                            "normal": "#2E89FC"
                        },
                        "good": {
                            "normal": "#54a254"
                        },
                        "warning": {
                            "normal": "#e69500"
                        },
                        "attention": {
                            "normal": "#cc3300"
                        }
                    }
                }
            },
            "imageSizes": {
                "default": 40,
                "small": 40,
                "medium": 80,
                "large": 160
            },
            "imageSet": {
                "imageSize": "medium",
                "maxImageHeight": 100
            },
            "actions": {
                "color": "",
                "backgroundColor": "red",
                "maxActions": 5,
                "spacing": "extraLarge",
                "buttonSpacing": 20,
                "actionsOrientation": "horizontal",
                "actionAlignment": "center",
                "showCard": {
                    "actionMode": "inline",
                    "inlineTopMargin": 16,
                    "style": "default"
                }
            },
            "spacing": {
                "default": 5,
                "small": 5,
                "medium": 10,
                "large": 20,
                "extraLarge": 40,
                "padding": 20
            },
            "separator": {
                "lineThickness": 1,
                "lineColor": "#eeeeee"
            },
            "factSet": {
                "title": {
                    "size": "default",
                    "color": "default",
                    "weight": "bold",
                    "warp": True
                },
                "value": {
                    "size": "default",
                    "color": "default",
                    "weight": "default",
                    "warp": True
                },
                "spacing": 20
            },
            "supportsInteractivity": True,
            "imageBaseUrl": "",
            "height": 743
        },
        "currentCardName": "Multiple Indicator",
        "narration": {
            "enabled": False,
            "display": "above",
            "format": "bullets",
            "verbosity": "medium",
            "up_sentiment": "good",
            "aggregation": "sum",
            "labels": []
        }
    }
}
    payload_columns = generate_payload(widget_features)
    payload['style']['currentCard']['body'][0]['items'][1]['columns'] = payload_columns['columns']
    # dash_endpoint = f'{url}/api/v1/dashboards/{dash_id}/widgets'
    response = sisense_conn.call_api_custom('POST',url,widget_endpoint, params=None, payload=payload)
    # response = requests.post(widget_endpoint, headers=source_head, data=json.dumps(payload))
    widget_id = response.json()["oid"]
    
    if response.status_code == 201:
        print(f"Widget '{widget_id}' - '_auto_sklearn_online_prediction' in '{dashboard_name}' created successfully")
    else:
        print(f'Failed to create Widget in "{dashboard_name}". Status code: {response.status_code}')




### Create result dataframe for model name and accuracy score

In [ ]:
# Extract metric name and value from score
key, value = next(iter(score.items()))
df = pd.DataFrame()
df['model_name'] = [model]
df['metric_name'] = [key]
df['score'] = [round(value, 2)]
path = folder_path + '/'
df['path'] = [path]
df

### Save model information in CSV

In [ ]:
folder_path = f"/opt/sisense/storage/notebooks/custom_code_notebooks/notebooks/automl/models/"
os.makedirs(folder_path, exist_ok=True)
new_df = df.copy()
new_df['provider'] = 'Auto-Sklearn'
new_df['created_date'] = datetime.now()
new_df['dash_id'] = dash_id
new_df['widget_id'] = widget_id
new_df

In [ ]:
models_file_path = f"/opt/sisense/storage/notebooks/custom_code_notebooks/notebooks/automl/models/models.csv"
# Check if models.csv file exists
if not os.path.exists(models_file_path):
    # If it does not exist, create an empty DataFrame and save it as models.csv
    empty_df = pd.DataFrame(columns=new_df.columns)
    empty_df.to_csv(models_file_path, index=False)
    print('empty')
old_df = pd.read_csv(models_file_path)
if old_df.empty:
    combined_df = new_df
else:
    combined_df = pd.concat([old_df, new_df], axis=0)

combined_df.to_csv(models_file_path,index=False)

### Save model information in Custom Code Table

In [ ]:
df_result = pd.DataFrame(df)